In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from helper_functions.preprocessing import replace_black_with_median, preprocess_image_v1
from helper_functions.data_transformation import get_transformed_data
from models.resnet import HybridResNet50, ResNet50
from models.basic_cnn import CharDataset


# Paths
TRAIN_FOLDER_PATH = "../data/train/combine"
TEST_FOLDER_PATH = "../data/test/combine"

In [2]:
# CNN parameters
BATCH_SIZE = 64
EPOCHS = 15
VALIDATION_SPLIT = 0.2

In [4]:
img = cv2.imread('../data/train/combine/000s-0.png')
processed_img = preprocess_image_v1(img)

cv2.imshow('image', processed_img)

# Wait for a key press (0 means indefinite wait)
cv2.waitKey(0)

# Close all OpenCV windows
cv2.destroyAllWindows()

# Load Training Data

In [3]:
print("Preparing training data...")
X_train_img, y_train_onehot, train_feature_names, X_train_features, train_scaler = get_transformed_data(TRAIN_FOLDER_PATH)
y_train = torch.tensor(np.argmax(y_train_onehot, axis=1), dtype=torch.long)

Preparing training data...


Preparing Data: 100%|█████████████████████████████████████████████████████████████| 7437/7437 [00:31<00:00, 239.29it/s]


In [4]:
# Create dataloaders
train_loader = DataLoader(CharDataset(X_train_img, y_train), batch_size=256, shuffle=True)

# RESNET Model


In [5]:
model = ResNet50()

In [5]:
model(torch.from_numpy(X_img[0]).unsqueeze(0), torch.from_numpy(X_features[0]).unsqueeze(0))

torch.Size([1, 2112])


tensor([[ 0.1171, -0.0208, -0.1096,  0.0038, -0.0432,  0.0911, -0.1665,  0.0548,
          0.0095,  0.0318, -0.1229,  0.0141,  0.0483,  0.1041,  0.0208,  0.1567,
         -0.0105, -0.1032,  0.0182,  0.0535,  0.0118,  0.2354,  0.0773,  0.0660,
         -0.0843, -0.0902,  0.0165,  0.0968, -0.0620, -0.0428, -0.0486,  0.0984,
          0.0055,  0.0326,  0.1354, -0.0027]], grad_fn=<AddmmBackward0>)

In [14]:
print(type(X_features[0]))

<class 'numpy.ndarray'>


In [9]:
model.train_model(train_loader)
torch.save(model, "./saved_models/resnet50.pth")

Epoch [1/1], Loss: 0.4262
Training complete!


# Hybrid RESNT Model

In [4]:
class HybridDataset(Dataset):
    def __init__(self, X_img, X_features, y):
        self.X_img = torch.tensor(X_img, dtype=torch.float32)
        self.X_features = torch.tensor(X_features, dtype=torch.float32)
        self.y = y

    def __len__(self):
        return len(self.X_img)

    def __getitem__(self, idx):
        return (self.X_img[idx], self.X_features[idx]), self.y[idx]


In [5]:
# Create dataloaders
hybrid_train_loader = DataLoader(HybridDataset(X_train_img, X_train_features, y_train), batch_size=256, shuffle=True)

In [ ]:
hybrid_model = HybridResNet50()
hybrid_model.train_model(hybrid_train_loader)
torch.save(model, "./saved_models/hyrbridresnet50.pth")

torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256, 2112])
torch.Size([256,